# Generate LPCs From Audio Files

In [27]:
import librosa
import numpy as np

## Load In Audio Files into Memory

In [28]:
# Sample Rate is 44.1 kHz
RECORDED_SAMPLE_RATE = 44100

# Desired Sample Rate for LPC Processing is 14 kHz
DESIRED_SAMPLE_RATE = 14000

DOWN_SAMPLE_COEFFICIENT = RECORDED_SAMPLE_RATE // DESIRED_SAMPLE_RATE

print(DOWN_SAMPLE_COEFFICIENT)

3


In [46]:
audio_file, original_sr= librosa.load(
    "./wav_data/vowels_Maria/aa_maria.wav",
    sr=RECORDED_SAMPLE_RATE)

print(audio_file[0:10])
# Add back in high frequencies using the heuristic that 
# sampled audio points may 


down_sampled_audio_file = librosa.resample(
    audio_file,
    orig_sr=original_sr,
    target_sr=DESIRED_SAMPLE_RATE)
print()
print(down_sampled_audio_file[0:10])

[0.         0.         0.         0.00015259 0.00109863 0.0038147
 0.00735474 0.00854492 0.00778198 0.00823975]

[-0.00010162  0.00057011  0.00683768  0.00844336  0.00733297  0.00714927
  0.00704003  0.00769634  0.0079391   0.00793288]


In [36]:
print(audio_file)
print(len(audio_file))
# print(len(down_sampled_audio_file))

[-0.06699143 -0.12399124 -0.11376429 ... -0.00111862 -0.00384832
 -0.00448074]
4203


In [31]:
lpc_vals = librosa.lpc(audio_file, order=14)

In [32]:
print(lpc_vals)
len(lpc_vals)

[ 1.         -2.0502448   1.6862448  -1.8335428   2.2697132  -1.8342547
  1.2652496  -0.8695163   0.794616   -0.6137525   0.40180787 -0.40603107
  0.19580787  0.042658   -0.04086584]


15